# RAG로 AI소믈리에 wine pairing

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력 : 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체 사용하기

In [3]:
# 함수 정의
def desccribe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
    You are a culinary expert who analyzes food images.
    
    When a user provides an image of a dish:

    - “Identify the commonly accepted name of the dish.”

    - “Describe the flavor profile clearly and concisely, focusing on the cooking method, texture, aroma, and balance of taste.
    If there is uncertainty, base your judgment on the most likely dish, avoid definitive statements, and maintain a professional tone.”
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래의 이미지 요리에 대한 요리명과 요리의 풍미를 설명해 주세요.
            출력형태:
            요리명:
            요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )

    output_parser = StrOutputParser()

    # LCEL (Langchain Expression Language)
    chain = prompt | llm | output_parser

    return chain

In [4]:
from langchain_core.runnables import RunnableLambda


# 함수 전달 인자로 func <- 함수 넣기
r1 = RunnableLambda(desccribe_dish_flavor)

input_data = {"image_url" :"https://www.sbfoods-worldwide.com/ko/recipes/deq4os00000008l9-img/10_Stake_A.jpg"}
res = r1.invoke(input_data)

In [5]:
print(res)

요리명: 스테이크 (또는 구운 채소를 곁들인 스테이크)

요리의 풍미:
이 요리는 팬에 구워졌거나 그릴에 조리된 것으로 보이는 스테이크가 주를 이루며, 겉은 노릇하게 시어링 되어 풍부한 육향과 캐러멜화된 풍미를 선사하고, 속은 촉촉하고 부드러운 육즙이 가득할 것으로 예상됩니다. 미디엄 레어에서 미디엄 굽기로 보이는 단면은 부드러운 식감을 암시합니다.

곁들여진 구운 아스파라거스와 버섯은 각각 신선하고 아삭한 식감과 촉촉하고 쫄깃한 식감, 그리고 흙내음 가득한 고유의 향을 더하며, 구운 토마토는 살짝 터지면서 상큼한 단맛과 산미를 제공하여 전체 요리의 균형을 잡아줍니다. 로즈마리 허브는 은은하게 퍼지는 향긋하고 쌉쌀한 아로마로 식욕을 돋우며, 스테이크의 깊은 맛을 더욱 풍성하게 할 것입니다.

사진상의 녹색 소스(와사비나 허브 페이스트로 추정)는 스테이크의 기름진 맛을 개운하게 잡아주거나, 신선한 허브의 향을 더해 전체적으로 savory함과 fresh함이 조화를 이루는 복합적인 맛의 경험을 제공할 것으로 보입니다.


# 요리에 가장 잘어울리는 wine top 5 검색
- pinecone 벡터 db 저장되어 있음
- index : wine-reviews, namespace : wine-reviews-ns

In [7]:
# 사용자 프롬프트 vector화, 유사도 높은 top-5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# 요리에 대한 풍미(설명) 들어오면,
# 벡터 db에 인덱싱할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 벡터 생성
embedding = OpenAIEmbeddings(
    model = OPENAI_EMBEDDING_MODEL
)

# 벡터 db에서 유사도계산, top-5 검색
vector_db = PineconeVectorStore(
    embedding = embedding ,
    index_name = PINECONE_INDEX_NAME ,
    namespace = PINECONE_NAMESPACE
)
# 벡터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 질문
print("질문: " , query)
print("-"*50)
vector_db.similarity_search(query, k=5) # top-5 검색

질문:  요리명: 스테이크 (또는 구운 채소를 곁들인 스테이크)

요리의 풍미:
이 요리는 팬에 구워졌거나 그릴에 조리된 것으로 보이는 스테이크가 주를 이루며, 겉은 노릇하게 시어링 되어 풍부한 육향과 캐러멜화된 풍미를 선사하고, 속은 촉촉하고 부드러운 육즙이 가득할 것으로 예상됩니다. 미디엄 레어에서 미디엄 굽기로 보이는 단면은 부드러운 식감을 암시합니다.

곁들여진 구운 아스파라거스와 버섯은 각각 신선하고 아삭한 식감과 촉촉하고 쫄깃한 식감, 그리고 흙내음 가득한 고유의 향을 더하며, 구운 토마토는 살짝 터지면서 상큼한 단맛과 산미를 제공하여 전체 요리의 균형을 잡아줍니다. 로즈마리 허브는 은은하게 퍼지는 향긋하고 쌉쌀한 아로마로 식욕을 돋우며, 스테이크의 깊은 맛을 더욱 풍성하게 할 것입니다.

사진상의 녹색 소스(와사비나 허브 페이스트로 추정)는 스테이크의 기름진 맛을 개운하게 잡아주거나, 신선한 허브의 향을 더해 전체적으로 savory함과 fresh함이 조화를 이루는 복합적인 맛의 경험을 제공할 것으로 보입니다.
--------------------------------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}